<a href="https://colab.research.google.com/github/coldbilberry/repo-gui/blob/main/%D0%92%D0%B2%D0%B5%D0%B4%D0%B5%D0%BD%D0%B8%D0%B5_%D0%B2_%D0%BE%D0%B1%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%BA%D1%83_%D0%B5%D1%81%D1%82%D0%B5%D1%81%D1%82%D0%B2%D0%B5%D0%BD%D0%BD%D0%BE%D0%B3%D0%BE_%D1%8F%D0%B7%D1%8B%D0%BA%D0%B0_%D0%A3%D1%80%D0%BE%D0%BA_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
!mkdir data

In [14]:
from google.colab import files

upload = files.upload()
!mv 'evgenyi_onegin.txt' 'data/evgenyi_onegin.txt'

MessageError: ignored

In [15]:
!ls data/*.txt

ls: cannot access 'data/*.txt': No such file or directory


In [12]:
path_to_file = '/content/data/evgenyi_onegin.txt'

In [ ]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

In [ ]:
print(text[:500])

In [ ]:
# text = text + text

In [ ]:
text = text.split('\n\n')

In [ ]:
text[0:5]

In [ ]:
len(text)

In [ ]:
text_only = []

for line in text:
    if len(line) < 350:
        continue
    else:
        text_only.append(line)

len(text_only)

In [ ]:
text_only[1]

In [ ]:
data = pd.DataFrame(text_only)
data = data.rename(columns={0: "text"})
data.head(3)

In [ ]:
def exclude_punctuation(txt):
    txt = "".join(c for c in txt if c not in exclude)
    txt = re.sub("\n", " \n", txt)
    return txt

In [ ]:
# data_1 = data

In [ ]:
# !pip install pymorphy2
!pip install stop-words

In [ ]:
from string import punctuation
from stop_words import get_stop_words

exclude = set(punctuation)
sw = set(get_stop_words("ru"))

In [ ]:
data['text_splited'] = data['text'].apply(exclude_punctuation)

In [ ]:
data.head(3)

In [ ]:
def preprocess_text(txt):
    txt = str(txt)
    txt = txt.lower()
    txt = re.sub("\n", "aaa", txt)
    new_txt =[]
    for word in txt.split():
        if word == "aaa":
            word = " \n"
        new_txt.append(word)

    return new_txt

In [ ]:
data['text_splited'] = data['text_splited'].apply(preprocess_text)

In [ ]:
data

In [ ]:
# Создадим словарь наших текстов
vocab = []
for ts in data.text_splited:
    for t in ts:
        vocab.append(t)

# dictionary = [ t for ts in df.text_splited for t in ts ]

vocab[:5]

In [ ]:
len(vocab)

In [ ]:
from collections import Counter

# Подсчитать частоту слов в списке и отсортировать по частоте
counts = Counter(vocab)
# counts.items()
print(dict(list(counts.items())[:5]))

In [ ]:
# Сортировка по частоте
sorted_counts = sorted(counts.items(), key=lambda item: (-item[1]))
sorted_counts[:5]

In [ ]:
# Частотный словарь
freq_dictionary = list(tp[0] for tp in sorted_counts)
freq_dictionary[:10]

In [ ]:
def get_w2i_i2w(column_data):

    dump = list(column_data.values)
    dump_txt_split = []
    for sublist in dump:
        for item in sublist:
            dump_txt_split.append(item)


    vocab = sorted(set(dump_txt_split))

    # Creating a mapping from unique characters to indices
    word2idx = {u:i for i, u in enumerate(vocab)}
    idx2word = np.array(vocab)

    print(len(vocab), len(word2idx), len(idx2word))
    return word2idx, idx2word

In [ ]:
# word2idx = {u:i for i, u in enumerate(vocab)}
# idx2word = np.array(vocab)

word2idx, idx2word = get_w2i_i2w(data['text_splited'])
data['int_text_splited'] = data['text_splited'].apply(lambda x: [word2idx[c] for c in x])

In [ ]:
# word2idx

In [ ]:
idx2word

In [ ]:
data.head(3)

In [ ]:
def get_all_int(column_data):

    values = column_data.values
    text_as_int = []
    print(values[0])
    for sublist in values:
        for item in sublist:
            text_as_int.append(item)
    text_as_int = np.array(text_as_int)

    return text_as_int

text_as_int = get_all_int(data['int_text_splited'])

In [ ]:
# text_as_int

In [ ]:
import numpy as np

unique, counts = np.unique(text_as_int, return_counts=True)

result = np.column_stack((unique, counts))
print (result)

In [ ]:
# The maximum length sentence you want for a single input in characters
seq_length = 50
examples_per_epoch = len(text_as_int)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(10):
    print(idx2word[i.numpy()])

In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
sequences

In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(' '.join(idx2word[input_example.numpy()])))
    print('Target data:', repr(' '.join(idx2word[target_example.numpy()])))

In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(idx2word)

# The embedding dimension
embedding_dim = 128

# Number of RNN units
rnn_units = 1024

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),

        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [ ]:
# def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
#     model = tf.keras.Sequential([
#         tf.keras.layers.Embedding(vocab_size, embedding_dim,
#                                   batch_input_shape=[batch_size, None]),

#         tf.keras.layers.LSTM(rnn_units,
#                             return_sequences=True,
#                             stateful=True,
#                             recurrent_initializer='glorot_uniform'),

#         tf.keras.layers.LSTM(rnn_units,
#                             return_sequences=True,
#                             stateful=True,
#                             recurrent_initializer='glorot_uniform'),

#          tf.keras.layers.LSTM(rnn_units,
#                             return_sequences=True,
#                             stateful=True,
#                             recurrent_initializer='glorot_uniform'),

#         tf.keras.layers.LSTM(rnn_units,
#                             return_sequences=True,
#                             stateful=True,
#                             recurrent_initializer='glorot_uniform'),

#         tf.keras.layers.Dense(vocab_size)
#     ])
#     return model

In [ ]:
model = build_model(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

In [ ]:
model.summary()

In [ ]:
example_batch_predictions[0]

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

In [ ]:
# print(idx2word[input_example_batch[0]])
print("Input: \n", repr(" ".join(idx2word[input_example_batch[0]])))
print()
# print("Next Word Predictions: \n", repr("".join(idx2word[8426])))
print("Next Word Predictions: \n", repr(" ".join(idx2word[sampled_indices])))

In [ ]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    # save_freq=88*5,
    period=20,
    save_weights_only=True)

In [ ]:
EPOCHS = 200

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [ ]:
model.summary()

In [ ]:
def generate_text(model, start_string, tmprt):
    # Evaluation step (generating text using the learned model)

    start_string = exclude_punctuation(start_string)
    #print(start_string)
    start_string_asis = preprocess_text(start_string)

    # Number of characters to generate
    num_generate = 30

    # Converting our start string to numbers (vectorizing)
    input_eval = [word2idx[s] for s in start_string_asis]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    # temperature = 0.1
    temperature = tmprt

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2word[predicted_id])

    return (start_string + ' '.join(text_generated))

In [ ]:
print(generate_text(model, start_string=u"зима ", tmprt=1.3))

In [ ]:
start_string= 'Холодных чистых как зима '

In [ ]:
print(generate_text(model, start_string=start_string, tmprt=1))

In [ ]:
print(generate_text(model, start_string=start_string, tmprt=0.5))

In [ ]:
print(generate_text(model, start_string=start_string, tmprt=5))